Sử dụng thuật toán di truyền để tìm trọng số cho mạng nơ ron tế bào cho bài toán tìm biên ảnh.
Một số lưu ý trong quá trình code:
- Do hàm giải phương trình vi phân (ODE) chỉ nhận mảng 1 chiều (array) vào giá trị ban đầu cũng như đầu ra của x nên phải biến đổi từ dạng ma trận về mảng 1 chiều ở một số phép tính.
- Thứ tự của m, n (kích thước của ma trận đầu vào) đôi khi bị ngược nhau trong quá trình reshape. Lưu ý khi đưa vào ma trận không vuông.
- Các số dùng trong phép tính nên để ở dạng float

In [6]:
from scipy.signal import convolve2d
from scipy.integrate import ode
from PIL import Image as img
import numpy as np
import numpy.matlib as mat

In [7]:
# hàm trạng thái
def hamTrangThai(t, x, u, B, A, I, n, m):
    x = x.reshape(n, m)
    dx = -x + convolve2d(u, B, 'same') + convolve2d(hamDauRa(x), A, 'same') + I
    return dx.reshape(n*m)

# hàm đầu ra
def hamDauRa(x):
    return 0.5 * (abs(x + 1) - abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    arr2[0], arr2[2] = arr2[2], arr2[0]#nếu không có lệnh này thì ma trận tạo ra sẽ bị ngược
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Hàm lai hai số
def lai(a,b,k=1):#k là điểm lai (mặc định là 1)  67|8
    return np.round(a/10**k)*10**k+b%10**k

# Hàm tạo số ngẫu nhiên từ 0 đến 2000
def ngauNhien(x=2000):
    return np.random.randint(0,x)

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

# Quần thể trọng số (gồm 11 số)
quanThe = np.array([ngauNhien() for i in range(11)])

# kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
m, n = 5, 5

# ma trận ảnh đầu vào u
u = np.array([
    [ 1, 1, 1, 1,-1],
    [ 1, 1, 1, 1,-1],
    [ 1, 1, 1, 1, 1],
    [ 1, 1, 1, 1, 1],
    [-1,-1, 1, 1, 1]], dtype=float)

# ma trận đầu ra mong muốn d
d = np.array([
    [ 1, 1, 1, 1,-1],
    [ 1,-1,-1, 1,-1],
    [ 1,-1,-1,-1, 1],
    [ 1, 1,-1,-1, 1],
    [-1,-1, 1, 1, 1]], dtype=float)

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1

# ma trận điều khiển B ban đầu
# B = np.array([
#     [-1.,-1.,-1.],
#     [-1., 8.,-1.],
#     [-1.,-1.,-1.]])
B = taoMaTranDoiXung(quanThe[0:5])

# ma trận phản hồi A ban đầu
# A = np.array([
#     [ 0., 0., 0.],
#     [ 0., 1., 0.],
#     [ 0., 0., 0.]])
A = taoMaTranDoiXung(quanThe[5:10])

# mức ngưỡng I ban đầu
I = quanThe[10]

# sai số cho phép
E0 = 2.0

# hai giá trị ban đầu cho x với t0 = 0 và t để tính đầu ra x (t != t0)
x, t = u*0.0, 10.0 # cần đưa x0 về dạng mảng một chiều

# ma trận đầu ra ban đầu
y = hamDauRa(x)

# khởi tạo phương trình vi phân
ptvp = ode(hamTrangThai) \
.set_integrator('vode') \
.set_initial_value(x.flatten(), 0.0) \
.set_f_params(u, B, A, I, n, m)

print(A)
print(B)

[[1320 1053 1657]
 [1548 1531 1053]
 [1657 1548 1320]]
[[1341 1443 1337]
 [ 480  830 1443]
 [1337  480 1341]]


In [10]:
#Tìm trọng số
Etruoc = 1000000000 # sai số của lần tính trước đó
S = 5 # số cha mẹ được chọn để lai mới mỗi trọng số 
loop = 100 #số lần chạy thử tối đa
count = 0 # số lần chạy
while(ptvp.successful() and Etruoc>E0 and count<loop):
    for i in range(5):
        tsl = count%11 # STT trọng số hiện tại đang được lai (trọng số lai)
        while(1):
            trongSoMoi = lai(quanThe[tsl],ngauNhien())
            if(trongSoMoi!=quanThe[tsl]):
                break
        trongSoCu = quanThe[tsl]
        quanThe[tsl] = trongSoMoi
        A = taoMaTranDoiXung(quanThe[0:5])
        B = taoMaTranDoiXung(quanThe[5:10])
        I = quanThe[10]
        ptvp.set_f_params(u, B, A, I, n, m)
        x = (ptvp.integrate(t))[:].reshape(n, m)
        y = hamDauRa(x)
        E = saiSo(d,y)
        if(E>=Etruoc):
            quanThe[tsl] = trongSoCu
        else:
            Etruoc = E
    
    # đếm số lần chạy
    count+=1
    print(f'Lần chạy thứ {count}: E = {Etruoc}')
    

Lần chạy thứ 1: E = 22.0
Lần chạy thứ 2: E = 22.0
Lần chạy thứ 3: E = 22.0
Lần chạy thứ 4: E = 22.0
Lần chạy thứ 5: E = 22.0
Lần chạy thứ 6: E = 22.0
Lần chạy thứ 7: E = 22.0
Lần chạy thứ 8: E = 22.0
Lần chạy thứ 9: E = 22.0
Lần chạy thứ 10: E = 22.0
Lần chạy thứ 11: E = 22.0
Lần chạy thứ 12: E = 22.0
Lần chạy thứ 13: E = 22.0
Lần chạy thứ 14: E = 22.0
Lần chạy thứ 15: E = 22.0
Lần chạy thứ 16: E = 22.0
Lần chạy thứ 17: E = 22.0
Lần chạy thứ 18: E = 22.0
Lần chạy thứ 19: E = 22.0
Lần chạy thứ 20: E = 22.0
Lần chạy thứ 21: E = 22.0
Lần chạy thứ 22: E = 22.0
Lần chạy thứ 23: E = 22.0
Lần chạy thứ 24: E = 22.0
Lần chạy thứ 25: E = 22.0
Lần chạy thứ 26: E = 22.0
Lần chạy thứ 27: E = 22.0
Lần chạy thứ 28: E = 22.0
Lần chạy thứ 29: E = 22.0
Lần chạy thứ 30: E = 22.0
Lần chạy thứ 31: E = 22.0
Lần chạy thứ 32: E = 22.0
Lần chạy thứ 33: E = 22.0
Lần chạy thứ 34: E = 22.0
Lần chạy thứ 35: E = 22.0
Lần chạy thứ 36: E = 22.0
Lần chạy thứ 37: E = 22.0
Lần chạy thứ 38: E = 22.0
Lần chạy thứ 39: E = 

Test các hàm

In [28]:
taoMaTranDoiXung([1,2,3,4,5])
taoMaTranDoiXung([np.random.randint(100,999) for i in range(5)])

array([[911, 600, 149],
       [553, 646, 600],
       [149, 553, 911]])

In [7]:
saiSo(d,y)

4.0

In [1]:
# khởi tạo phương trình vi phân
ptvp = ode(hamTrangThai) \
.set_integrator('vode') \
.set_initial_value(x.flatten(), 0.0) \
.set_f_params(u, B, A, I, n, m)
assert ptvp.successful()
x = ptvp.integrate(t)
y = hamDauRa(x[:].reshape(n, m))
y

NameError: name 'saiSo' is not defined

In [12]:
lai(123,456,2)

156.0

In [19]:
laiTrongSo(A,B,I)

(array([[135., 668., 279.],
        [267., 652., 668.],
        [279., 267., 135.]]),
 array([[932., 453., 555.],
        [ 55., 467., 453.],
        [555.,  55., 932.]]),
 -1.0)

In [24]:
np.random.randint(100,999)

533